# [Module 2.1] 컴프리핸드 토픽 모델링 요청

이 노트북은 아래와 같은 작업을 합니다.
- 전처리 데이터의 로딩 및 리뷰 컬럼만 추출하여 S3에 업로드
    - 토픽 모델링 job 요청시 입력으로 사용 됩니다.
- 컴프리핸드에 토픽 모델링 요청
- 토픽 모델링 결과 파일 위치 저장

In [1]:
import pandas as pd

In [2]:
%store -r grocery_negative_path

## 전처리 데이터 로딩 및 전처리 컬럼만 S3에 업로드

In [3]:
df = pd.read_csv(grocery_negative_path)
doc_df = df.review_body_kr_cl

In [4]:
import sagemaker
from utils import upload_s3
import os

bucket = sagemaker.Session().default_bucket()
prefix = 'ml-architect/topic'

cp_data_folder = 'preproc_data/comprehend'
os.makedirs(cp_data_folder, exist_ok=True)

grocery_negative_path = os.path.join(cp_data_folder, 'amazon_review_grocery_negative.csv')
doc_df.to_csv(grocery_negative_path, index=None)
s3_grocery_negative_path = upload_s3(bucket, grocery_negative_path, prefix)
print("s3_grocery_negative_path: \n", s3_grocery_negative_path)



s3_grocery_negative_path: 
 s3://sagemaker-ap-northeast-2-057716757052/ml-architect/topic/preproc_data/comprehend/amazon_review_grocery_negative.csv


## Comprehend에 Topic Modeling 요청
- 컴프리핸드에 토픽 모델링 요청을 합니다. 
- **[중요]** 아래를 실행하기 전에 반드시 SageMaker Execution Role에 ComprehendFullAccess, IAMFullAcess 추가를 해야 합니다. 그렇지 않을 경우에는 에러가 발생 합니다.

In [5]:
import boto3
import boto3
import numpy as np
import sagemaker

bucket = sagemaker.Session().default_bucket()
role = 'arn:aws:iam::057716757052:role/service-role/AmazonComprehendServiceRoleS3FullAccess-gsmoon'
region_name='ap-northeast-2'

# Instantiate Boto3 SDK:
client = boto3.client('comprehend', region_name= region_name)


def reqest_topic_comprehend(number_of_topics, 
                        s3_uri,
                        s3_output_path,
                        job_name,                            
                        client):
    '''
    comprehend에 학습 요청
    '''
    start_response = client.start_topics_detection_job(
        NumberOfTopics = number_of_topics,      
        JobName = job_name,
        InputDataConfig={
            'S3Uri': s3_uri,
            'InputFormat': 'ONE_DOC_PER_LINE'
        },
        OutputDataConfig={
            'S3Uri': s3_output_path
        },
        DataAccessRoleArn= role
    )
    
    return start_response
    

## Topic Modeling 실행
중요 파라미터로서 **number_of_topics 를 넣어주세요.(현재 : 3)**

In [6]:
# 중요: S3 입력 파일 경로
# s3_input_uri = s3_negative_path
s3_input_uri = s3_grocery_negative_path


# S3 결과 파일 저장 경로 및 기타

cp_topic_prefix = 'ml-architect/comprehend/output'
s3_cp_topic_output_path = "s3://{}/{}".format(bucket, cp_topic_prefix)
print("s3_cp_topic_output_path: \n", s3_cp_topic_output_path)

import numpy as np
suffix = str(np.random.uniform())[4:9]

job_name = 'cp_topic_' + suffix
#################################
number_of_topics = 3
#################################
cp_topic_response = reqest_topic_comprehend(
                        number_of_topics,  # 모델 식별자
                        s3_input_uri, # 입력 위치
                        s3_cp_topic_output_path, # 출력 위치
                        job_name,                        
                        client )

describe_response = client.describe_topics_detection_job(JobId = cp_topic_response['JobId'])


s3_cp_topic_output_path: 
 s3://sagemaker-ap-northeast-2-057716757052/ml-architect/comprehend/output


## 토픽 모델링 결과 위치 저장
컴프리핸드는 토픽 모델링 후에 결과를 S3에 저장 합니다. 이 위치를 저장하여 다음 노트북에서 사용합니다.
- **아래의 job_status 가 `SUBMITTED` 가 있으면 기다려야 합니다.**
    - **이 시간은 약 30분 소요 됩니다.**
    - Comprehend의 콘솔에 가서 확인이 가능합니다.
    - 아래 그림에서 Stauts 가 'Completed' 로 되어야 합니다.
    - ![2.comprehend-consloe](img/2.comprehend-consloe.png)

In [14]:
s3_output_path = describe_response['TopicsDetectionJobProperties']['OutputDataConfig']['S3Uri']
print("s3_output_path: \n", s3_output_path)
job_status = describe_response['TopicsDetectionJobProperties']['JobStatus']
job_status

s3_output_path: 
 s3://sagemaker-ap-northeast-2-057716757052/ml-architect/comprehend/output/057716757052-TOPICS-0762f27bf6904bec139d51e52897eb65/output/output.tar.gz


'SUBMITTED'

## S3 결과 경로 저장

In [15]:
%store s3_output_path

Stored 's3_output_path' (str)
